In [1]:
import collections
import math
import os
import random
import zipfile
import datetime as dt
import numpy as np
import tensorflow as tf

In [2]:
# 输入训练路径
train_path = '/home/jiang/data/sorted10000'
# 基站路径
station_path = '/home/jiang/data/stationRecord'
# 输入长度
maxlen = 150
# 输入中某地（如海口）大于该值，进入训练集
threshold = 120
# 滚动步长
step = 1
# 测试长度
testlen = 50
batch_size=16
num_skips=2
skip_window=2
data_index1=0
data_index2=0
station_size=200
lr=0.01

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

num_steps = 2000

station_size = 10000
embedding_size = 300

In [3]:
def read_data(train_path):
    file_object = open(train_path)
    train_list = file_object.readlines()
    train_set=[]
    user_record=[]
    station_list=[]
    station_set=set()
    for i,record in enumerate(train_list):
        record=record.strip()
        user,station=(record.split(",")[j] for j in [0,3])
        station_set.add(int(station))
        if i == len(train_list)-1:
            user_record.append(int(station))
            train_set.append(user_record)
            station_list=sorted(list(station_set))
            return train_set,station_list
        elif user != train_list[i+1].split(",")[0]:
            user_record.append(int(station))
            train_set.append(user_record)
            user_record=[]
        else:
            user_record.append(int(station))
    #return train_set,station_list

In [4]:
#注意不要让序列小于3个的序列进入
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index1
    global data_index2
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    #print len(data[data_index1])
    for i in range(batch_size // num_skips):
        #第一次进入循环初始化buffer
        if i == 0:
            while len(data[data_index1])-data_index2 < span:
                data_index2 = 0
                data_index1 = (data_index1 + 1)%len(data)                
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        ###############################
        #index更新后判断是否越界
        if data_index2 == (len(data[data_index1])):
            data_index2 = 0
            data_index1 = (data_index1 + 1)%len(data)
            while len(data[data_index1])-data_index2 < span:
                data_index1 = (data_index1 + 1)%len(data)
            for _ in range(span-1):
                buffer.append(data[data_index1][data_index2])
                data_index2 += 1
        #################################
        #print data_index1,data_index2 
        buffer.append(data[data_index1][data_index2])
        #print buffer
        data_index2 += 1
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    #data_index2 = (data_index2 + len(data) - span) % len(data)
    return batch, context

In [5]:
def recount_data(data,station_indices):
    new_data=[]
    for record in data:
        new_record=[]
        for station in record:
            new_record.append(station_indices[station])
        new_data.append(new_record)
    return new_data

In [6]:
def data_pre(train_path):
    train_data,station_data=read_data(train_path)
    sum_station = len(station_data)
    station_indices = dict((c, i) for i, c in enumerate(station_data))
    indices_station = dict((i, c) for i, c in enumerate(station_data))
    train_data = recount_data(train_data,station_indices)
    return train_data,sum_station,indices_station

In [7]:
data,station_num,indices_station=data_pre(train_path)
print station_num

1183


In [8]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Look up embeddings for inputs.
    embeddings = tf.Variable(
      tf.random_uniform([station_num, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([station_num, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([station_num]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=station_num))

    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(nce_loss)
    
#   # Construct the variables for the softmax
#     weights = tf.Variable(tf.truncated_normal([embedding_size, station_num],
#                                               stddev=1.0 / math.sqrt(embedding_size)))
#     biases = tf.Variable(tf.zeros([station_num]))
#     hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases

#   # convert train_context to a one-hot format
#     train_one_hot = tf.one_hot(train_context, station_num)

#     cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))

#   # Construct the SGD optimizer using a learning rate of lr.
#     optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)

  # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
    init = tf.global_variables_initializer()

In [9]:
session=tf.Session(graph=graph)
average_loss = 0
softmax_start_time = dt.datetime.now()
for step in range(num_steps):
    session.run(init)
    batch_inputs, batch_context = generate_batch(data,batch_size, num_skips, skip_window)
    #print batch_inputs,batch_context
    feed_dict = {train_inputs: batch_inputs, train_context: batch_context}
    #print feed_dict
    _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
    #_, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
    average_loss += loss_val
    #print step
    if step % 10 == 0:
        if step > 0:
            average_loss /= 10
    # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 200 == 0:
        sim = similarity.eval(session=session)
        for i in range(valid_size):
            valid_station = indices_station[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_station
            for k in range(top_k):
                close_station = indices_station[nearest[k]]
                log_str = '%s %s,' % (log_str, close_station)
            print(log_str)
final_embeddings = normalized_embeddings.eval(session=session)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run".format((softmax_end_time-softmax_start_time).total_seconds()))

Nearest to 10673: 10802, 43473, 11583, 43842, 42591, 30363, 13742, 12391,
Nearest to 10491: 38154, 11672, 19282, 11522, 12402, 43513, 11383, 31352,
Nearest to 10522: 13791, 13221, 18251, 19261, 15313, 14732, 11763, 13524,
Nearest to 10062: 18061, 14452, 12553, 23451, 38154, 24211, 10351, 14754,
Nearest to 10773: 11211, 14811, 12083, 38159, 11722, 44043, 42501, 15662,
Nearest to 10242: 12263, 12531, 11053, 14751, 19172, 13951, 22036, 10643,
Nearest to 10106: 21421, 42321, 43512, 13713, 43053, 16235, 17022, 14732,
Nearest to 10333: 42421, 42461, 11811, 24262, 13843, 40303, 43221, 11522,
Nearest to 10202: 10011, 42521, 40303, 15313, 19933, 13762, 47571, 39156,
Nearest to 10661: 17012, 12023, 13303, 13093, 10683, 31932, 31691, 13571,
Nearest to 10221: 11073, 44231, 11783, 11303, 13561, 47541, 12651, 21482,
Nearest to 10642: 24386, 48311, 11001, 21492, 30922, 12481, 12002, 11003,
Nearest to 10331: 42773, 13243, 13562, 42221, 10223, 11781, 44041, 18564,
Nearest to 10782: 13571, 48311, 13851,

('Average loss at step ', 620, ': ', 149.24222564697266)
('Average loss at step ', 630, ': ', 151.1774932861328)
('Average loss at step ', 640, ': ', 150.27618408203125)
('Average loss at step ', 650, ': ', 151.46022796630859)
('Average loss at step ', 660, ': ', 145.04606323242189)
('Average loss at step ', 670, ': ', 150.24979858398439)
('Average loss at step ', 680, ': ', 147.97459411621094)
('Average loss at step ', 690, ': ', 147.29170074462891)
('Average loss at step ', 700, ': ', 146.66934967041016)
('Average loss at step ', 710, ': ', 145.6235321044922)
('Average loss at step ', 720, ': ', 150.06959381103516)
('Average loss at step ', 730, ': ', 145.04517974853516)
('Average loss at step ', 740, ': ', 149.83752899169923)
('Average loss at step ', 750, ': ', 150.62796325683593)
('Average loss at step ', 760, ': ', 151.07276000976563)
('Average loss at step ', 770, ': ', 150.31548309326172)
('Average loss at step ', 780, ': ', 150.98615722656251)
('Average loss at step ', 790, ':

('Average loss at step ', 1420, ': ', 155.26338500976561)
('Average loss at step ', 1430, ': ', 147.68453521728514)
('Average loss at step ', 1440, ': ', 145.86368255615236)
('Average loss at step ', 1450, ': ', 154.43888549804689)
('Average loss at step ', 1460, ': ', 147.3147003173828)
('Average loss at step ', 1470, ': ', 147.86567840576171)
('Average loss at step ', 1480, ': ', 145.98232269287109)
('Average loss at step ', 1490, ': ', 146.05819854736328)
('Average loss at step ', 1500, ': ', 155.90278472900391)
('Average loss at step ', 1510, ': ', 144.52509613037108)
('Average loss at step ', 1520, ': ', 151.37565307617189)
('Average loss at step ', 1530, ': ', 148.39995422363282)
('Average loss at step ', 1540, ': ', 149.00134887695313)
('Average loss at step ', 1550, ': ', 149.27793884277344)
('Average loss at step ', 1560, ': ', 150.00018157958985)
('Average loss at step ', 1570, ': ', 149.68550415039061)
('Average loss at step ', 1580, ': ', 148.85749969482421)
('Average loss 